In [ ]:
# But first, let's install the dependecies.
!pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
from video_prompter import get_connection

# TODO: setup .env file
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

# connect to VideoDB
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url=url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ run the prompt on video --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

In [2]:
from videodb import play_stream

# ----Existing video case ----

# TODO: replace with your video id
# # video_id = "m-replace-with-your-video-id-24-7"
# video_id = 'm-beb3ee4a-3169-4cb6-8dc1-beee73eb00a1'
# video_id = 'm-beb3ee4a-3169-4cb6-8dc1-beee73eb00a1'
video_id = "m-2c2b792f-424c-432b-bcbe-9e3c7e00dcbf"


# ---- Fresh video case ----
# url = "https://youtu.be/pAMy7IhOVQE?si=6HMxVGP13hAHtmW3"
# video = fresh_video(url)
# video_id = video.id


#watch the original video
video = conn.get_collection().get_video(video_id)
video.player_url

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/71ad9a7c-6b05-46f8-8248-ed314cc94e9f.m3u8'

In [4]:
scene = video.list_scene_index()
print(scene)

[{'name': 'Scene Index 2024-07-22 07:52', 'scene_index_id': '5cbd6d6fb68a8baa', 'status': 'done'}, {'name': 'Scene Index 2024-07-22 08:39', 'scene_index_id': '7e5d8456aba295af', 'status': 'done'}, {'name': 'Scene Index 2024-07-22 09:02', 'scene_index_id': 'bf7802c6cbe5d3b1', 'status': 'done'}]


In [69]:
# Create new scene index
from videodb import IndexType
from videodb import SceneExtractionType
 
index_id = video.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time":10, "select_frames": ['first']},
    prompt="all the actions taken by charchter in the frame",
    # callback_url=callback_url,
)

# Wait to Indexing to finish
# scene_index = video.get_scene_index()
# print(scene_index)

# #Use existing scene index


# scene_index = video.get_scene_index("bc32304c34e5e5a6")
# print(scene_index)


In [3]:
scene_index_id = video.list_scene_index()
print(scene_index_id)

[{'name': 'Scene Index 2024-07-22 07:52', 'scene_index_id': '5cbd6d6fb68a8baa', 'status': 'done'}, {'name': 'Scene Index 2024-07-22 08:39', 'scene_index_id': '7e5d8456aba295af', 'status': 'done'}, {'name': 'Scene Index 2024-07-22 09:02', 'scene_index_id': 'bf7802c6cbe5d3b1', 'status': 'done'}]


In [4]:
scene_index = video.get_scene_index("bf7802c6cbe5d3b1")
print(scene_index)

[{'description': 'In the frame, there is a character who appears to be performing the following actions:\n\n1. The character is partially visible, standing behind a doorway.\n2. The character is shining a flashlight into the room or hallway, illuminating the space.\n\nThese actions suggest that the character could be searching for something or someone, or possibly investigating an area.', 'end': 10.0, 'start': 0.0}, {'description': 'The image shows a close-up of a cordless phone with an answering machine. No character is visible in the frame, so no actions taken by a character can be described.', 'end': 20.0, 'start': 10.0}, {'description': "In this frame, the character is holding a phone up to their ear, indicating they are listening or speaking to someone on the phone. The person's expression suggests they may be deep in thought or concentration, but no other actions are clearly visible.", 'end': 30.0, 'start': 20.0}, {'description': 'In the frame, the character is performing the fol

In [14]:
from video_prompter import scene_prompter


# scene_index = video.get_scene_index(index_id)
#Choose a prompt to create create clip. 
# user_prompt = "all the gun shots fired"
# user_prompt = "standing behind a doorway"
user_prompt = "talking on phone"
result = scene_prompter(scene_index, user_prompt)
print(result)

Sendiing call to OPENAI 
        You are a video editor who uses AI. Given a user prompt and AI-generated scene descriptions of a video, analyze the descriptions to identify segments relevant to the user prompt for creating clips.

        - **Instructions**: 
            - Evaluate the scene descriptions for relevance to the specified user prompt.
            - Ensure that selected segments have clear start and end points, covering complete ideas or actions.
            - Choose segments with the highest relevance and most comprehensive content.
            - Optimize for engaging viewing experiences, considering visual appeal and narrative coherence.
            - If closely related descriptions exist, combine them into a single, cohesive segment.
            - Each selected segment should be substantial enough for a meaningful clip, typically covering at least 5 to 10 seconds of video content.

            - User Prompts: Interpret prompts like 'find exciting moments' or 'identify k

In [8]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb import IndexType
from videodb.asset import VideoAsset, AudioAsset, ImageAsset
from videodb.timeline import Timeline
# from videodb.asset import TextAsset, TextStyle

timeline = Timeline(conn)
for description in result:
    search_res = video.search(description, index_type =IndexType.scene)
    print(search_res)
    matched_segments = search_res.get_shots()
    print(len(matched_segments),"length mat")
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    #video segment
    video_shot = matched_segments[0]

    print(video_shot.start,video_shot.end )

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

SearchResult(collection_id=c-50e11372-632c-40b0-8aa7-c055d8d89b8d, stream_url=None, player_url=None, shots=[Shot(video_id=m-2c2b792f-424c-432b-bcbe-9e3c7e00dcbf, video_title=None, start=185.0, end=186.0, text=The character in the frame is engaged in several actions:

1. The character is talking on a phone, bringing it close to their ear and mouth.
2. The character appears to be seated in an office chair.
3. The character seems to have raised their left hand towards their face.

The setting indicates motion, potentially suggesting urgency or a tense situation. There may also be a dynamic light source, suggesting an active environment., search_score=0.639981508, stream_url=None, player_url=None)])
1 length mat
185.0 186.0
SearchResult(collection_id=c-50e11372-632c-40b0-8aa7-c055d8d89b8d, stream_url=None, player_url=None, shots=[Shot(video_id=m-2c2b792f-424c-432b-bcbe-9e3c7e00dcbf, video_title=None, start=220.0, end=230.0, text=In the frame, the character is in the following actions:

1. 

In [9]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/c9bc9f26-9649-4828-b5ac-5a20b9fa15c8.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/c9bc9f26-9649-4828-b5ac-5a20b9fa15c8.m3u8'

In [16]:
from videodb.timeline import Timeline


def construct_timeline_and_stream( selected_frames):
    timeline = Timeline(conn)
    
    print("Selected frames:", selected_frames)  

    for frame in selected_frames:
        print("Adding frame:", frame)  
        video_asset = VideoAsset(
            asset_id=video_id,
            start=frame['start_time'],
            end=frame['end_time']
        )
        timeline.add_inline(video_asset)
    
    print("Timeline created:", timeline)  
    
    stream_url = timeline.generate_stream()
    print("Stream URL:", stream_url)
    return play_stream(stream_url)


In [17]:
from video_prompter import extract_timestamps,merge_timeframes
from videodb.timeline import Timeline


timeframes = extract_timestamps(result)
merged_timeframes = merge_timeframes(timeframes)
construct_timeline_and_stream(merged_timeframes)

Extracted timeframe: start=20, end=30
Extracted timeframe: start=35, end=45
Extracted timeframe: start=150, end=160
Appending: {'start_time': 35, 'end_time': 45}
Appending: {'start_time': 150, 'end_time': 160}
Selected frames: [{'start_time': 20, 'end_time': 30}, {'start_time': 35, 'end_time': 45}, {'start_time': 150, 'end_time': 160}]
Adding frame: {'start_time': 20, 'end_time': 30}
Adding frame: {'start_time': 35, 'end_time': 45}
Adding frame: {'start_time': 150, 'end_time': 160}
Timeline created: <videodb.timeline.Timeline object at 0x109150970>
Stream URL: https://stream.videodb.io/v3/published/manifests/aa863872-ed64-4425-9c05-4a2d49902a60.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/aa863872-ed64-4425-9c05-4a2d49902a60.m3u8'

In [75]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/e6a005de-e685-47f8-8ad0-a1a1206d55e7.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/e6a005de-e685-47f8-8ad0-a1a1206d55e7.m3u8'